### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize,PowerNorm
import matplotlib.cm as cm
from tqdm.auto import tqdm
import sys
import os 
import warnings

warnings.filterwarnings("ignore")


C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Custom

In [2]:
sys.path.append(os.path.abspath('./PINNS/'))
sys.path.append(os.path.abspath('./PDE/'))
sys.path.append(os.path.abspath('./Losses/'))
from Poisson2D import *
from PoissonCircLoss import CircLoss
from IPINN import IPINN
from VDLNet import VDLNet

In [3]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Plotting Functions

In [4]:
def compare_contours(X,Y,exact,result,colormap = 'plasma',levels=10,nrange = (0,1)):
    """
    Produces a contour plot comparing exact and predicted results
    X,Y: Axes in form of meshgrids
    exact: Analytical Solution
    result: PINN solution
    It will also return the RMSE and L2 errors 
    """
    res = np.sqrt(np.sum(np.square(exact - result)))/np.sqrt(np.sum(np.square(exact)))
    rmse = np.sqrt(np.mean(np.square(exact- result)))
    err = np.abs(exact - result)
    
    fig, axes = plt.subplots(1, 3,figsize=(18, 4))
    cmap = cm.get_cmap(colormap)
    normalizer = Normalize(nrange[0],nrange[1])
    im = cm.ScalarMappable(norm = normalizer,cmap = colormap)
    fig.colorbar(im, ax=axes.ravel().tolist())

    axes[0].contourf(X, Y, result, levels=levels, cmap=colormap,norm = normalizer)
    axes[0].set_title('Prediction')

    axes[1].contourf(X, Y, exact , levels=levels, cmap=colormap,norm = normalizer)
    axes[1].set_title('Actual')
    
    axes[2].contourf(X, Y, err , levels=levels, cmap=colormap,norm=normalizer)
    axes[2].set_title('RMSE:{:.2e}, Rel. L2:{:.2e}'.format(rmse,res))
    
    plt.show()

    res2 = np.sum(err)
    print('L1: ',res2)

In [5]:
def plot_results3d(X,Y,exact,result,colormap = 'cividis'):
    """
    Produces a surface plot comparing exact,predicted results and error between them
    X,Y: Axes in form of meshgrids
    exact: Analytical Solution
    result: PINN solution
    """
    fig,axes = plt.subplots(1,3,figsize=(18, 4),subplot_kw={'projection': '3d'})
    error = exact - result
    
    axes[0].plot_surface(X, Y, result, cmap=colormap)
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_zlabel('u')
    axes[0].set_title('Predicted')
    
    axes[1].plot_surface(X, Y, exact, cmap=colormap)
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    axes[1].set_zlabel('u')
    axes[1].set_title('Analytical Solution')
    
    axes[2].plot_surface(X, Y, error, cmap=colormap)
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('y')
    axes[2].set_zlabel('Error')
    axes[2].set_title('Error')
    
    plt.show()

## Two Dimensional Elliptical Interface Problems

### 2. Poisson equation with circular interface

- $Ω=[−1,1]×[−1,1]$ with a  interface:
- $x(\theta) = (a+b \cos (m \theta)) \sin (n \theta) \cos (\theta)$
- $y(\theta) = (a+b \cos (m \theta)) \sin (n \theta) \sin (\theta)$
- $\theta$ in $[0,2\pi]$ $a=b=0.40178$，$m=2$,$n=6$

The PDE is:
- $-∇⋅(κ_m∇u_m)=f(x)$ in $Ω_m$
- $u=g$  on $∂Ω$
- $〚u〛=  \varphi$ on $Γ_{int}$
- $〚κ∇u〛⋅\mathbf{n_2}=\psi$ on $Γ_{int}$

where $∂Ω^d_2=\{x:x=−1∪x=1∪y=−1∪y=1\}$ is the external boundary. 

The coefficients k_m and the solutions u_m:
\begin{equation}
k = \left\{
\begin{array}{ll}
\frac{x^{2} - y^{2} + 3}{7}, & x \in \Omega_{1},\\
\frac{2 + xy}{5}, & x \in \Omega_{2}, 
\end{array}
\right.
\quad
u = \left\{
\begin{array}{ll}
\sin(x + y) + \cos(x + y) + 1, & x \in \Omega_{1}, \\
x + y + 1, & x \in \Omega_{2}.
\end{array}
\right.
\end{equation}



In [6]:
#Set the parameters
#Get loss function
# 定义 K1 和 K2 函数
def K1(x, y):
    return (x**2-y**2+3)/7

def K2(x, y):
    return (2+x*y)/5

# 创建 CircLoss 类的实例并传入 K1 和 K2 函数
ls_circ = CircLoss(n_points=10000, int_pts=300, bound_pts=200, K=[K1, K2])
#Get the exact function
circ = Poisson2DCirc()

In [7]:
h = 0.001
x = torch.arange(-1, 1+h, h)
y = torch.arange(-1, 1+h, h)
X, Y = torch.meshgrid(x, y)
XY = torch.stack((X, Y), dim=-1).reshape(-1, 2).to(device)
def getResCirc(outputs):
    u1,u2 = outputs[0],outputs[1]
    result = torch.zeros_like(u1).cpu().numpy().reshape(X.shape)
    u1 = u1.cpu().numpy().reshape(X.shape)
    u2 = u2.cpu().numpy().reshape(X.shape)
    theta = torch.arctan2(Y,X)
    theta = torch.where(theta < 0, theta + 2 * torch.pi, theta)  # 将角度调整到[0, 2π]范围内
    # 计算每个点对应的花瓣半径
    x_petal = (0.40178 + 0.40178 * torch.cos(2 * theta) * torch.sin(6 * theta)) * torch.cos(theta)
    y_petal = (0.40178 + 0.40178 * torch.cos(2 * theta) * torch.sin(6 * theta)) * torch.sin(theta)
    mask1 = X**2 + Y**2 <x_petal**2 + y_petal**2
    mask2 =X**2 + Y**2 >x_petal**2 + y_petal**2
    result[mask1] = u1[mask1]
    result[mask2] = u2[mask2]
        
    return result

#### IPINN

In [8]:
def cond_func2(x,condition):
    if(condition == '0'):
        return F.silu(x)
    elif(condition == '1'):
        return F.tanh(x)

In [9]:
#define the model
ipinn2 = IPINN(dimension = 2,hidden_size = 10,depth = 3)
#Get the loss_function
ipinn2.setLoss(lambda x: ls_circ.loss(x,'ipinn',cond_func2))
#Train the model
ipinn2.train(iterations = 100000)

  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch:  0  Loss:  10.806547


  0%|          | 101/100000 [00:11<2:29:36, 11.13it/s]

Epoch:  100  Loss:  3.782047


  0%|          | 199/100000 [00:23<2:46:16, 10.00it/s]

Epoch:  200  Loss:  2.0202239


  0%|          | 263/100000 [00:30<2:03:55, 13.41it/s]

In [ ]:
activations = [F.silu,F.tanh]
#Plot the results
outputs = ipinn2.eval(XY,activations)
result = getResCirc(outputs)
exact_circ = circ.equation(X.cpu(),Y.cpu())
compare_contours(X.cpu(),Y.cpu(),exact_circ,result,colormap = 'cividis',levels=10,nrange=(0,2))

In [ ]:
plot_results3d(X.cpu(),Y.cpu(),exact_circ,result)

#### XPINN(VDL)

In [ ]:
#define the model
activations = [F.tanh,F.tanh,F.tanh]
xpinn2 = VDLNet(num_domains = 2,dimension = 2,activations = activations,hidden_size = 10,depth = 3)
#Get the loss_function
ls_circ.reset()
xpinn2.setLoss(lambda x: ls_circ.loss(x,'xpinn',cond_func2))
#Train the model
xpinn2.train(iterations = 10000)

In [ ]:
#Plot the results
outputs = xpinn2.eval(XY)
result = getResCirc(outputs)
compare_contours(X.cpu(),Y.cpu(),circ.equation(X.cpu(),Y.cpu()),result,colormap = 'plasma',levels=10,nrange=(0,2))

In [ ]:
plot_results3d(X.cpu(),Y.cpu(),exact_circ,result)

### Results

After running each model for an hour these were the results:

1. Poisson with a straight interface

Metric|IPINN|XPINN(VDL)
---|:---:|:---:
RMSE|$2.0 × 10^{-4}$|$3.1× 10^{-1}$  
Rel. L2|$2.4 \times 10^{-4}$|$3.7× 10^{-1}$

2. Poisson with a circular interface

Metric|IPINN|XPINN(VDL)
---|:---:|:---:
RMSE|$4.2 × 10^{-3}$|$7.4× 10^{-1}$
Rel. L2|$2.9 \times 10^{-3}$|$5.1× 10^{-1}$

All models had hidden layer size of $10$, depth of $3$ and were trained on $10^4$ points

IPINN works better in higher dimensions 